# Parameter tuning for SVC

This notebook was inspired by:  
- [In Depth: Parameter tuning for SVC](https://medium.com/all-things-ai/in-depth-parameter-tuning-for-svc-758215394769)
- [SVM Hyperparameter Tuning using GridSearchCV](https://www.vebuso.com/2020/03/svm-hyperparameter-tuning-using-gridsearchcv/):

The goal of this notebook is to show the basic influence of the kernel choice of a support vector classifier (SVC). Furthermore, the SVC parameters are described and their effect on the classification result is shown.  
For the sake of better illustration, the explanations are given using the famous [Iris flower data sets](https://en.wikipedia.org/wiki/Iris_flower_data_set) as an example.

## Load globally used libraries and set plot parameters

In [1]:
import time

from IPython.display import HTML

## About the dataset

The Iris flower data set is a multivariate data set introduced by Sir Ronald Fisher in the 1936 as an example of discriminant analysis.  
The data set consists of 50 samples from each of three species of Iris ([*Iris setosa*](https://en.wikipedia.org/wiki/Iris_setosa), [*Iris virginica*] and *Iris versicolor*), so there are 150 total samples. Four features were measured from each sample: the length and the width of the [sepals](https://en.wikipedia.org/wiki/Sepal) and [petals](https://en.wikipedia.org/wiki/Petal), in centimetres.  
Here is a principle illustration of a flower with sepal and petal:

In [21]:
display(HTML("<figure><img src='./images/Mature_flower_diagram.svg' width='800px'> \
                 <figcaption>Principle illustration of a flower with sepal and petal (source: <a href='https://en.wikipedia.org/wiki/File:Mature_flower_diagram.svg'>Mature_flower_diagram.svg</a>)</figcaption> \
              </figure>"))

Here are pictures of the three different Iris species (*Iris setosa*, *Iris virginica* and *Iris versicolor*). Given the dimensions of the flower, it will be possible to predict the class of the flower.

In [22]:
display(HTML("<table> \
                <tr> \
                <td><figure><img src='./images/Iris_setosa_640px.jpg' width='320px'> \
                        <figcaption><i>Iris setosa</i> (source: <a href='https://commons.wikimedia.org/wiki/File:Irissetosa1.jpg'>Irissetosa1.jpg</a>)</figcaption> \
                    </figure></td> \
                <td><figure><img src='./images/Iris_versicolor_640px.jpg' width='320px'> \
                        <figcaption><i>Iris versicolor</i> (source: <a href='https://en.wikipedia.org/wiki/File:Iris_versicolor_3.jpg'>Iris versicolor 3.jpg</a>)</figcaption> \
                    </figure></td> \
                <td><figure><img src='./images/Iris_virginica_590px.jpg' width='295px'> \
                        <figcaption><i>Iris virginica</i> (source: <a href='https://en.wikipedia.org/wiki/File:Iris_virginica.jpg'>Iris virginica.jpg</a>)</figcaption> \
                    </figure></td> \
                </tr> \
              </table>"))

Iris setosa (source: Irissetosa1.jpg),Iris versicolor (source: Iris versicolor 3.jpg),Iris virginica (source: Iris virginica.jpg)


The iris data set is widely used as a beginner's dataset for machine learning purposes.  
Among others, the dataset is included in Python in the machine learning package [Scikit-learn](https://scikit-learn.org), so that users can access it without having to find a special source for it.